In [1]:
import numpy as np
import cupy as cp
import pandas as pd
import networkx as nx
from numba import cuda
import time
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("CA-GrQc.csv")

In [3]:
df.head()

,FromNodeId,ToNodeId
0,3466,937
1,3466,5233
2,3466,8579
3,3466,10310
4,3466,15931


In [4]:
df.shape[0] # number of edges

28980

In [5]:
# making the graph
G = nx.from_pandas_edgelist(df, source='FromNodeId', target='ToNodeId')

In [ ]:
# draw the graph
'''
plt.figure(figsize=(100, 50))
nx.draw(
    G,
    with_labels=False,
    node_color='red',
    edge_color='black',
    node_size=2000,
    font_size=8
)
plt.title("Graph of LastFM User Network")
plt.show()
'''

In [6]:
def print_matrix(matrix):
    for row in matrix:
        print(" ".join(f"{val:.2f}" for val in row))

In [7]:
adj_lastfm = nx.to_numpy_array(G, dtype=np.float32)

print("Numpy Adjacency Matrix:\n", adj_lastfm)

Numpy Adjacency Matrix:
 [[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


In [8]:
def Normalize(matrix):
    matrix2 = matrix.copy()

    for j in range(len(matrix2[0])):
        temp = 0
        for i in range(len(matrix2)):
            temp += matrix2[i][j]
        for i in range(len(matrix2)):
            matrix2[i][j] = matrix2[i][j] / temp
        
    return matrix2

def Inflate(matrix, r):
    matrix = cp.power(matrix, r)
    col_sums = cp.sum(matrix, axis=0)
    matrix = matrix / col_sums
    return matrix

@cuda.jit
def matmul(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.0
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

def powerMatrix(matrix, n, bpg, tpb):
    if n < 1:
        raise ValueError("Pangkat matrix harus >= 1")

    matrix1 = matrix
    matrix2 = cp.zeros_like(matrix)

    for i in range(n - 1):
        matmul[bpg, tpb](matrix1, matrix, matrix2)
        cuda.synchronize()  # penting untuk sinkronisasi kernel
        matrix1 = matrix2.copy()

    return matrix1

def print_matrix(matrix):
    for row in matrix:
        print(" ".join(f"{val:.2f}" for val in row))

def MarkovCluster(matrix_a, e, r, iter):
    matrix = matrix_a

    # Add self-loops
    for i in range(len(matrix)):
        matrix[i][i] = 1

    matrix = Normalize(matrix)

    print("Normalized Matrix:\n", matrix)

    matrix = cp.asarray(matrix)

    # Setup CUDA kernel execution config
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(matrix.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(matrix.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    iterate = iter
    tol = 1e-7
    ptr = 0
    converged = False

    while ptr < iterate and not converged:
        prev_matrix = matrix.copy()

        matrix = powerMatrix(matrix, e, bpg=blockspergrid, tpb=threadsperblock)
        matrix = Inflate(matrix, r)

        # Cek konvergensi
        diff = cp.abs(matrix - prev_matrix)
        max_diff = cp.max(diff)
        if max_diff < tol:
            converged = True

        ptr += 1

    matrix = matrix.get()
    return matrix

In [9]:
e_1 = 2
r_1 = 2
start = time.time()
MarkovMatrix = MarkovCluster(matrix_a=adj_lastfm, e = e_1, r = r_1, iter = 2)
end = time.time()

KeyboardInterrupt: 

In [16]:
print(MarkovMatrix)

[[1.0000000e+00 1.1613098e-14 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.3333334e-01
  3.3333334e-01 3.3333334e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.3333334e-01
  3.3333334e-01 3.3333334e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.3333334e-01
  3.3333334e-01 3.3333334e-01]]


In [17]:
def get_clusters_from_square_matrix(mcl_matrix):
    """
    mcl_matrix: square numpy array (n_nodes x n_nodes), hasil dari MCL
                dengan 1s menunjukkan koneksi antar node dalam cluster
    return: list of clusters, each cluster is a list of node indices
    """
    # Buat graph dari adjacency matrix
    G = nx.from_numpy_array(mcl_matrix)

    # Ambil connected components (cluster)
    clusters = [list(component) for component in nx.connected_components(G)]

    return clusters

In [18]:
clusters = get_clusters_from_square_matrix(MarkovMatrix)

for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}: {cluster}")

Cluster 1: [0, 1, 3, 5, 7, 8, 2060, 1066, 1068, 1070, 1071, 1072, 1074, 1077, 1078, 1083, 1084, 2111, 2123, 2124, 2125, 2126, 2127, 88, 606, 620, 4245, 2218, 2219, 2220, 2221, 2222, 740, 741, 742, 743, 746, 747, 749, 750, 751, 755, 756, 3338, 3341, 3342, 4418, 4419, 4420, 4421, 4422, 4438, 373, 2962, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1459, 1460, 1461, 1463, 1976, 1977, 1978, 1979, 1980, 1465, 1464, 449, 450, 451, 452, 453, 455, 456, 458, 459, 460, 461, 465, 466, 1491, 1492, 1493, 468, 471, 467, 1494, 474, 1496, 476, 477, 478, 479, 1497, 1498, 482, 504]
Cluster 2: [2, 18, 4, 14]
Cluster 3: [1793, 1794, 1795, 1796, 1797, 6, 1798, 2195, 3237, 3239, 2224, 2225, 2226, 2227, 2228, 2992, 2993, 3386, 2787, 1790, 1791]
Cluster 4: [9, 3730, 3731, 3732]
Cluster 5: [10, 5072, 1649, 4634, 4635]
Cluster 6: [384, 385, 386, 387, 388, 389, 515, 391, 11, 139, 2835, 406, 407, 408, 409, 411, 1180, 414, 415, 2465, 2466, 2600, 2601, 2602, 2603, 2604, 2605, 2606, 570, 572, 574, 575, 576, 579, 5